In [ ]:
import pandas as pd
import PrepDataHelper as helper
import statsmodels.api as sm
import statsmodels.formula.api as smf
print(pd.__version__)
print(sm.__version__)

In [ ]:
physio_file = 'full_logfile_scr_3.csv'
df = pd.read_csv(physio_file)
df = df.drop(df.loc[df.rt_fear>df.rt_fear.mean()+3*df.rt_fear.std()].index)

df['hp_change_video']=df['video_hp']-df['base_ECG']
df['scr_change_video']=df['video_scr']-df['base_scr']
df = helper.TakeAwayGroupsLessThenX(df, min_groups=12)
# calculate jitter time

df['PreTaskJitterDuration']= df.video_start - (df.word_end + df.rt_exp_fear+df.rt_current_anxiety)

In [ ]:
standardized_fear_scr_hp  = df[['hp_change_video','participant_num','scr_change_video','resp_fear']].groupby('participant_num').transform(lambda x: (x - x.mean()) / x.std())
standardized_df = pd.concat([standardized_fear_scr_hp, df[['participant_num','video_condition','PreTaskJitterDuration']]], axis=1)
#only keep social (2) and spider conditions (3) 
standardized_df = standardized_df.loc[df.video_condition.isin([2,3])]
full_df_no_heights = standardized_df

# Frequentist Hierarchical

In [ ]:
md = smf.mixedlm("PreTaskJitterDuration ~ hp_change_video",
                 full_df_no_heights, 
                 groups=full_df_no_heights['participant_num'], missing='drop')
mdf = md.fit()
mdf.summary()

In [ ]:
md = smf.mixedlm("PreTaskJitterDuration ~ scr_change_video",
                 full_df_no_heights, 
                 groups=full_df_no_heights['participant_num'], missing='drop')
mdf = md.fit()
mdf.summary()

In [ ]:
md = smf.mixedlm("PreTaskJitterDuration ~ resp_fear",
                 full_df_no_heights, 
                 groups=full_df_no_heights['participant_num'], missing='drop')
mdf = md.fit()
mdf.summary()